In [5]:
import numpy as np
import pandas as pd
import os
POSTGRES_PASSWORD = os.getenv('POSTGRES_PASSWORD')
import psycopg2
from sqlalchemy import create_engine
import geopandas as gpd
from shapely import wkt
import fiona
import geoplot as gplt
import geoalchemy2
import geodatasets
import matplotlib
import folium

In [6]:
states_id = gpd.read_file("states/states_id.shp")
counties_id = gpd.read_file("counties/counties_us.shp")
beaches_id = gpd.read_file("beaches_id/beaches_id.shp")
sites_id = gpd.read_file("sites_id/sites_id.shp")
states_volume = pd.read_csv("states_volume/states_volume.csv")
reefs = gpd.read_file('reefs_info/ArtificialReef.gpkg')

In [3]:
reefs['reefYear'] = reefs['reefYear'].fillna(0).astype(int)

# Replace -1 back with NaN (if you want to preserve NaN values)
reefs['reefYear'].replace(-1, pd.NA, inplace=True)
def is_number(s):
    try:
        float(s)
        return True
    except ValueError:
        return False

reefs['hover_name'] = reefs['reefName'].apply(lambda x: 'NA' if pd.isna(x) or is_number(x) or x == 'NA' else x)

reefs.columns = [x.lower() for x in reefs.columns]
reefs.to_file('reefs_info/ArtificialReef.gpkg', mode = 'w')

KeyError: 'reefYear'

In [ ]:
states_id.columns = [x.lower() for x in states_id.columns]

In [ ]:
states_id.to_file('states/states_id.shp', mode = 'w')

In [ ]:
counties_id.columns = [x.lower() for x in counties_id.columns]
counties_id.head(2)

In [ ]:
counties_id.to_file('counties/counties_us.shp', mode = 'w')

## Create the Database

In [7]:
dbserver = psycopg2.connect(
    host = 'postgres',
    user = 'postgres',
    password = POSTGRES_PASSWORD,
    port = 5432
)
dbserver.autocommit = True
cursor = dbserver.cursor()

In [ ]:
try:
    cursor.execute('CREATE DATABASE tides')
    print("'tides' database created successfully.")
except:
    cursor.execute('DROP DATABASE tides')
    cursor.execute('CREATE DATABASE tides')

In [8]:
dbserver = psycopg2.connect(
    host = 'postgres',
    user = 'postgres',
    password = POSTGRES_PASSWORD,
    port = 5432,
    dbname='tides')

dbserver.autocommit = True
cursor = dbserver.cursor()

try:
    cursor.execute('CREATE EXTENSION IF NOT EXISTS postgis')
    print('PostGIS extension created successfully.')
except Exception as e:
    print(f'Error creating PostGIS extension: {e}')


PostGIS extension created successfully.


In [9]:
engine = create_engine('postgresql+psycopg2://{user}:{password}@{host}:{port}/{db}'.format(
    user = 'postgres',
    password = POSTGRES_PASSWORD,
    host = 'postgres',
    port = 5432,
    db = 'tides'
))

In [10]:
print(states_id.shape[0])
states_id.to_postgis('states_id', con=engine, index=False, chunksize=1000, if_exists = 'replace')

51


In [11]:
print(counties_id.shape[0])
counties_id.to_postgis('counties_id', con=engine, index=False, chunksize=1000, if_exists = 'replace')

3143


In [12]:
print(beaches_id.shape[0])
beaches_id.to_postgis('beaches_id', con=engine, index=False, chunksize=1000, if_exists = 'replace')

3789


In [13]:
print(sites_id.shape[0])
sites_id.to_postgis('sites_id', con=engine, index=False, chunksize=1000, if_exists = 'replace')

6575


In [14]:
print(states_volume.shape[0])
states_volume.to_sql('states_volume', con=engine, index=False, chunksize=1000, if_exists = 'replace')

3622


3622

In [15]:
print(reefs.shape[0])
reefs.to_postgis('reefs', con = engine, index = False, chunksize = 1000, if_exists = 'replace')

30398
